In [31]:
import pandas as pd
import numpy as np
import re


def transform_date(df, columns: str):
    df[columns] = pd.to_datetime(df[columns])
    # Extraire l'année, le mois et le jour de la semaine
    df['Années'] = df[columns].dt.year
    df['Mois'] = df[columns].dt.month
    df['Heure'] = df[columns].dt.hour
    df['Jour de la semaine'] = df[columns].dt.day_name()
    # Mapper les jours de la semaine en français
    jours_semaine_fr = {
        'Monday': 'LUNDI',
        'Tuesday': 'MARDI',
        'Wednesday': 'MERCREDI',
        'Thursday': 'JEUDI',
        'Friday': 'VENDREDI',
        'Saturday': 'SAMEDI',
        'Sunday': 'DIMANCHE'
    }
    # Mapper les mois en français
    mois_fr = {
        1: 'JANVIER',
        2: 'FEVRIER',
        3: 'MARS',
        4: 'AVRIL',
        5: 'MAI',
        6: 'JUIN',
        7: 'JUILLET',
        8: 'AOUT',
        9: 'SEPTEMBRE',
        10: 'OCTOBRE',
        11: 'NOVEMBRE',
        12: 'DECEMBRE'
    }
    # Remplacer les numéros de mois par leur équivalent en français
    df['Mois'] = df['Mois'].map(mois_fr)
    # Remplacer les noms des jours par leur équivalent en français
    df['Jour de la semaine'] = df['Jour de la semaine'].map(jours_semaine_fr)
    return df

def clean_cell_number(df, columns:str):
    df[columns] = df[columns].astype(str)
    df[columns] = df[columns].str.split(',').str[0]
    df[columns] = df[columns].replace(r'^0693', '262693', regex=True)
    df[columns] = df[columns].replace(r'^0692', '262692', regex=True)
    df[columns] = df[columns].replace(r'^06', '336', regex=True)
    df[columns] = df[columns].replace(r'^07', '337', regex=True)
    df[columns] = df[columns].replace(r'^02', '2622', regex=True)
    return df

def reset_accent(chaine):
    accent_fr = {
    'é': 'e', 'è': 'e', 'ê': 'e', 'ë': 'e',
    'à': 'a', 'â': 'a', 'ä': 'a',
    'ç': 'c',
    'î': 'i', 'ï': 'i',
    'ô': 'o', 'ö': 'o',
    'ù': 'u', 'û': 'u', 'ü': 'u',
    'ÿ': 'y',
    # Ajout des majuscules
    'É': 'E', 'È': 'E', 'Ê': 'E', 'Ë': 'E',
    'À': 'A', 'Â': 'A', 'Ä': 'A',
    'Ç': 'C',
    'Î': 'I', 'Ï': 'I',
    'Ô': 'O', 'Ö': 'O',
    'Ù': 'U', 'Û': 'U', 'Ü': 'U'
}
    for accent, sans_accent in accent_fr.items():
        chaine = chaine.replace(accent, sans_accent)
    return chaine


def clean_city(df, columns: str):
    df[columns] = df[columns].astype(str)
    df = df.rename(columns={columns: 'VILLE'})
    df['VILLE']= df['VILLE'].str.upper()
    df['VILLE']= df['VILLE'].str.replace("-", " ")
    df['VILLE']= df['VILLE'].str.replace("SAINT", "ST")
    df['VILLE']= df['VILLE'].str.replace("SAINTE", "STE")
    df['VILLE']= df['VILLE'].str.replace("L'", "")
    df['VILLE'] = df['VILLE'].apply(reset_accent)
    df['VILLE'] = df['VILLE'].str.replace(r'\s+', ' ', regex=True).str.strip()
    return df


def convert_date(date_str):
    # Extraire le fuseau horaire avec une expression régulière
    match = re.search(r'UTC([+-]\d)', date_str)
    if match:
        utc_offset = int(match.group(1))  # Récupérer le décalage horaire
    else:
        utc_offset = 0  # Valeur par défaut si aucun fuseau horaire n'est trouvé
    # Retirer le fuseau horaire et convertir en datetime
    date_without_tz = date_str.split(' UTC')[0]  # Retirer ' UTC+X'
    dt = pd.to_datetime(date_without_tz, format='%d/%m/%Y - %H:%M:%S')
    # Ajuster le fuseau horaire à UTC+4
    dt = dt + pd.Timedelta(hours=(4 - utc_offset))  # Ajustement basé sur l'offset
    return dt

# Fonction pour nettoyer les chaînes
def clean_number(number_str):
    # Utiliser une expression régulière pour supprimer le préfixe et le suffixe
    return re.sub(r'^\=\("\s*|\s*"\)$', '', number_str)

def replace_unknown_ville(row):
    reunion_postal_codes = {
    "97400": "ST DENIS",
    "97410": "ST PIERRE",
    "97411": "BOIS DE NEFLES",
    "97412": "BRAS PANON",
    "97413": "CILAOS",
    "97414": "ENTRE DEUX",
    "97416": "LA CHALOUPE",
    "97417": "ST BERNARD",
    "97418": "LA PLAINE DES CAFRES",
    "97419": "LA POSSESSION",
    "97420": "LE PORT",
    "97421": "LA RIVIERE ST LOUIS",
    "97422": "LA SALINE",
    "97423": "LE GUILLAUME",
    "97424": "PITON ST LEU",
    "97425": "LES AVRIONS",
    "97426": "LES TROIS BASSINS",
    "97427": "L'ETANG SALE",
    "97428": "LA NOUVELLE",
    "97429": "PETITE ILE",
    "97430": "LE TAMPON",
    "97431": "LA PLAINE DES PALMISTES",
    "97432": "LA RAVINE DES CABRIS",
    "97433": "SALAZIE - HELL BOURG",
    "97434": "ST GILLES LES BAINS",
    "97435": "BERNICA",
    "97436": "ST LEU",
    "97437": "STE ANNE",
    "97438": "STE MARIE",
    "97439": "STE ROSE",
    "97440": "ST ANDRE",
    "97441": "STE SUZANNE",
    "97442": "BASSE VALLEE",
    "97450": "ST LOUIS",
    "97460": "ST PAUL",
    "97470": "ST BENOIT"
}
    if row['VILLE'] == 'ville inconnue':
        return reunion_postal_codes.get(row['CODE POSTAL'], 'ville inconnue')  # Remplace par la ville correspondante ou garde 'ville inconnue'
    return row['VILLE']



In [82]:
import pandas as pd
import numpy as np
import re


def preprocess_data(file1):
    df = pd.read_csv(file1, sep=';', encoding='latin1', dtype={"CIBLE": str, "CORRESPONDANT": str, "DUREE": str, "IMSI": str, "IMEI": str, "CODE POSTAL": str,  "X": str, "Y": str, "VILLE": str, "ADRESSE2": str, "ADRESSE3": str, "ADRESSE4": str, "ADRESSE5": str})
    # Appliquer la fonction pour convertir les dates
    if 'DATE' in df.columns:
        df['DATE'] = df['DATE'].apply(convert_date)
        df = transform_date(df, 'DATE')
    if 'CORRESPONDANT' in df.columns:
        df['CORRESPONDANT'] = df['CORRESPONDANT'].fillna("DATA")
        df['CORRESPONDANT'] = df['CORRESPONDANT'].astype(str)
        df['CORRESPONDANT'] = df['CORRESPONDANT'].apply(clean_number)
        df = clean_cell_number(df, 'CORRESPONDANT')
    if 'DUREE' in df.columns:
        df["DUREE"] = df["DUREE"].fillna("0")
    if 'DIRECTION' in df.columns:
        df['DIRECTION'] = df['DIRECTION'].fillna("INDETERMINE")
        df['DIRECTION'] = df['DIRECTION'].str.upper()
    if 'IMEI' in df.columns:
        df['IMEI'] = df['IMEI'].fillna("INDETERMINE")
        df["IMEI"] = df["IMEI"].apply(clean_number)
    if 'IMSI' in df.columns:
        df["IMSI"] = df["IMSI"].astype('str')
        df['IMSI'] = df['IMSI'].apply(clean_number)
    if 'CIBLE' in df.columns:
        df['CIBLE'] = df['CIBLE'].apply(clean_number)
        df = clean_cell_number(df, 'CIBLE')
    if "TYPE" in df.columns:
        df["TYPE"] = df["TYPE"].astype(str)
        df["TYPE"] = df["TYPE"].str.upper()
        df["TYPE"] = df["TYPE"].apply(reset_accent)
    if 'VILLE' in df.columns:
        df['VILLE'] = df['VILLE'].fillna('INDETERMINE')
        df['VILLE'] = df.apply(replace_unknown_ville, axis=1)
        df = clean_city(df, columns='VILLE')
    if 'ADRESSE2' in df.columns:
        df['ADRESSE2'] = df['ADRESSE2'].fillna("INDETERMINE")
        df['CODE POSTAL'] = df['CODE POSTAL'].fillna("INDETERMINE")
     # Créer la colonne Adresse
    df["ADRESSE"] = np.where(
        df["ADRESSE2"] != "INDETERMINE",
        df["ADRESSE2"] + " " + df["CODE POSTAL"] + " " + df["VILLE"],
        'INDETERMINE'
    )
    # Nettoyer la colonne ADRESSE
    df['ADRESSE'] = df['ADRESSE'].str.replace(r'\s+', ' ', regex=True).str.strip()
    df['ADRESSE'] = df['ADRESSE'].str.upper()
    df['ADRESSE'] = df['ADRESSE'].apply(reset_accent)
    deleted_columns = ['TYPE CORRESPONDANT', 'COMP.', 'EFFICACITE' , 'CELLID', 'ADRESSE IP VO WIFI', 'PORT SOURCE VO WIFI', 'ADRESSE2','ADRESSE3','ADRESSE4', 'ADRESSE5', 'PAYS', 'TYPE-COORD', 'CODE POSTAL']
    df.drop(columns=deleted_columns, inplace=True, errors='ignore')
    rename_dict = {"TYPE": "TYPE D'APPEL", "CIBLE": "ABONNE", "X": "LATITUDE", "Y": "LONGITUDE", "converted_date": "DATE", "Années": "ANNEE", "Mois": "MOIS", "Heure": "HEURE", "Jour de la semaine": "JOUR DE LA SEMAINE"
    }
    df.rename(columns={k: v for k, v in rename_dict.items() if k in df.columns}, inplace=True)
    df.fillna("INDETERMINE", inplace=True)
    return df

In [83]:
df = preprocess_data("/home/antoine/telephony_insight_project/streamlit/app/source/models_files/TCOI_MT24.csv")

df.head(50)

,TYPE D'APPEL,DATE,ABONNE,CORRESPONDANT,DIRECTION,DUREE,IMSI,IMEI,VILLE,LATITUDE,LONGITUDE,ANNEE,MOIS,HEURE,JOUR DE LA SEMAINE,ADRESSE
0,TELEPHONIE,2024-06-14 09:04:44,262693856874,262262352253,SORTANT,63,647030000179983,866860053998200,CHALOUPE ST LEU,-21.1525,55.3179,2024,JUIN,9,VENDREDI,62 CHEMIN AGAPANTHES 97416 CHALOUPE ST LEU
1,DATA_PAQUET,2024-06-14 09:04:59,262693856874,DATA,SORTANT,68,647030000179983,866860053998200,CHALOUPE ST LEU,-21.1525,55.3179,2024,JUIN,9,VENDREDI,62 CHEMIN AGAPANTHES 97416 CHALOUPE ST LEU
2,DATA_PAQUET,2024-06-14 09:04:59,262693856874,DATA,SORTANT,68,647030000179983,866860053998200,CHALOUPE ST LEU,-21.1525,55.3179,2024,JUIN,9,VENDREDI,62 CHEMIN AGAPANTHES 97416 CHALOUPE ST LEU
3,TELEPHONIE,2024-06-14 11:18:51,262693856874,262692109747,ENTRANT,38,647030000179983,866860053998200,ST LEU,-21.153215,55.30459,2024,JUIN,11,VENDREDI,215 CHEMIN LAURENCY RIVIERE 97436 ST LEU
4,DATA_PAQUET,2024-06-14 11:18:53,262693856874,DATA,SORTANT,4,647030000179983,866860053998200,ST LEU,-21.153215,55.30459,2024,JUIN,11,VENDREDI,215 CHEMIN LAURENCY RIVIERE 97436 ST LEU
5,DATA_PAQUET,2024-06-14 11:18:53,262693856874,DATA,SORTANT,4,647030000179983,866860053998200,ST LEU,-21.153215,55.30459,2024,JUIN,11,VENDREDI,215 CHEMIN LAURENCY RIVIERE 97436 ST LEU
6,DATA_PAQUET,2024-06-14 11:19:09,262693856874,DATA,SORTANT,36,647030000179983,866860053998200,CHALOUPE ST LEU,-21.1525,55.3179,2024,JUIN,11,VENDREDI,62 CHEMIN AGAPANTHES 97416 CHALOUPE ST LEU
7,DATA_PAQUET,2024-06-14 11:19:09,262693856874,DATA,SORTANT,36,647030000179983,866860053998200,CHALOUPE ST LEU,-21.1525,55.3179,2024,JUIN,11,VENDREDI,62 CHEMIN AGAPANTHES 97416 CHALOUPE ST LEU
8,TELEPHONIE,2024-06-14 13:51:33,262693856874,262262352253,ENTRANT,53,647030000179983,866860053998200,ST LEU,-21.153215,55.30459,2024,JUIN,13,VENDREDI,215 CHEMIN LAURENCY RIVIERE 97436 ST LEU
9,DATA_PAQUET,2024-06-14 13:51:34,262693856874,DATA,SORTANT,4,647030000179983,866860053998200,ST LEU,-21.153215,55.30459,2024,JUIN,13,VENDREDI,215 CHEMIN LAURENCY RIVIERE 97436 ST LEU


In [61]:
import plotly.express as px
import pandas as pd

def plot_correspondant_bar(df):
    try:
        # 1. Compter le nombre total de communications par correspondant (sans tenir compte du type d'appel)
        correspondant_counts = df.groupby('CORRESPONDANT').size().reset_index(name='TOTAL_COMS')

        # 2. Filtrer pour garder uniquement les correspondants ayant 11 ou 12 caractères
        correspondant_counts = correspondant_counts[correspondant_counts['CORRESPONDANT'].str.len().isin([11, 12])]

        # 3. Trier par nombre total de communications et prendre les 10 premiers
        top_10_correspondants = correspondant_counts.sort_values(by='TOTAL_COMS', ascending=False).head(10)['CORRESPONDANT'].tolist()

        # 4. Filtrer le DataFrame original pour ne garder que les 10 premiers correspondants
        df_top_10 = df[df['CORRESPONDANT'].isin(top_10_correspondants)].copy()

        # 5. Grouper par correspondant ET type d'appel (maintenant qu'on a filtré les 10 premiers)
        grouped_counts = df_top_10.groupby(['CORRESPONDANT', 'TYPE D\'APPEL']).size().reset_index(name='NBRE COMS')

        # 6. Ajouter la colonne 'TOTAL_COMS' à grouped_counts
        grouped_counts = pd.merge(grouped_counts, correspondant_counts, on='CORRESPONDANT', how='left')

        # 7. Calculer le pourcentage du nombre de communications par rapport au nombre total de communications *pour chaque correspondant*
        grouped_counts['POURCENTAGE'] = ((grouped_counts['NBRE COMS'] / grouped_counts['TOTAL_COMS']) * 100).round(1)

        # 8. Trier grouped_counts par 'TOTAL_COMS' de manière décroissante (important avant de créer le graphique)
        grouped_counts = grouped_counts.sort_values(by='TOTAL_COMS', ascending=False)

        fig = px.bar(grouped_counts, x='CORRESPONDANT', y='NBRE COMS',
                     color='TYPE D\'APPEL',  # Ajout de la couleur en fonction du type d'appel
                     title='Nombre de Communications par Correspondant (Top 10)',
                     hover_data=['POURCENTAGE', 'TYPE D\'APPEL', 'TOTAL_COMS'],  # Inclure TOTAL_COMS dans les informations au survol
                     labels={'CORRESPONDANT': 'Correspondant', 'NBRE COMS': 'Nombre de Communications', 'TYPE D\'APPEL': 'Type d\'Appel', 'TOTAL_COMS': 'Total Communications'})

        # Mise en forme des barres
        fig.update_traces(
            marker_line_width=1,  # Ajout d'un contour noir
            marker_line_color="black",
            width=0.4,  # Réduction de la largeur des barres
            insidetextanchor="middle"
        )

        # Ajustement de la mise en page
        fig.update_layout(
            xaxis_tickangle=-45,  # Inclinaison des labels de l'axe X
            bargap=0.2  # Espacement entre les barres
        )

        return fig
    except Exception as e:
        print(f"Erreur lors de la création du graphique à barres: {e}")
        return None



fig = plot_correspondant_bar(df)
if fig:
    fig.show()

In [66]:
import plotly.graph_objects as go

def count_phone_type(df):
    try:
        type_appel_counts = df["TYPE D'APPEL"].value_counts().reset_index()
        type_appel_counts.columns = ['TYPE D\'APPEL', 'Nombre']
        total_coms = type_appel_counts['Nombre'].sum()  # Calculer le nombre total de communications
        colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']  # Définir les couleurs pour le graphique

        fig = go.Figure(data=[go.Pie(labels=type_appel_counts['TYPE D\'APPEL'],
                                       values=type_appel_counts['Nombre'],
                                       hole=.7)])  # Ajouter l'argument hole pour créer un donut

        fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                          marker=dict(colors=colors, line=dict(color='#000000', width=2)))

        # Ajouter une annotation au centre du donut pour afficher le nombre total de communications
        fig.update_layout(annotations=[dict(text=f'Total<br>{total_coms}', x=0.5, y=0.5, font_size=30, showarrow=False)])

        return fig  # Retourner la figure Plotly
    except Exception as e:
        print(f"Erreur lors du comptage des types d'appel: {e}")
        return go.Figure()  # Retourner une figure vide en cas d'erreur

fig = count_phone_type(df)

fig.show()

In [85]:
def plot_correspondant_bar(df):
    try:
        fig = px.bar(df, x='CORRESPONDANT', y='count', color ='TYPE D\'APPEL',
                     title='Nombre de Communications par Correspondant (Top 10)',
                     hover_data=['POURCENTAGE', 'TYPE D\'APPEL', 'TOTAL_COMS'],  # Inclure TOTAL_COMS dans les informations au survol
                     labels={'CORRESPONDANT': 'Correspondant', 'NBRE COMS': 'Nombre de Communications', 'TYPE D\'APPEL': 'Type d\'Appel'})
        # Ajustement de la mise en page
        fig.update_layout(
            xaxis_tickangle=-45,  # Inclinaison des labels de l'axe X
            bargap=0.2  # Espacement entre les barres
        )
        return fig
    except Exception as e:
        print(f"Erreur lors de la création du graphique à barres: {e}")
        return None

plot = plot_correspondant_bar(df)

plot.show()

Erreur lors de la création du graphique à barres: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ["TYPE D'APPEL", 'DATE', 'ABONNE', 'CORRESPONDANT', 'DIRECTION', 'DUREE', 'IMSI', 'IMEI', 'VILLE', 'LATITUDE', 'LONGITUDE', 'ANNEE', 'MOIS', 'HEURE', 'JOUR DE LA SEMAINE', 'ADRESSE'] but received: count


AttributeError: 'NoneType' object has no attribute 'show'

In [89]:
# 1. Compter le nombre total de communications par correspondant (sans tenir compte du type d'appel)
correspondant_counts = df.groupby('CORRESPONDANT').size().reset_index(name='TOTAL_COMS')

# 2. Filtrer pour garder uniquement les correspondants ayant 11 ou 12 caractères
correspondant_counts = correspondant_counts[correspondant_counts['CORRESPONDANT'].str.len().isin([11, 12])]

# 3. Trier par nombre total de communications et prendre les 10 premiers
top_10_correspondants = correspondant_counts.sort_values(by='TOTAL_COMS', ascending=False).head(10)['CORRESPONDANT'].tolist()

# 4. Filtrer le DataFrame original pour ne garder que les 10 premiers correspondants
df_top_10 = df[df['CORRESPONDANT'].isin(top_10_correspondants)].copy()

# 5. Grouper par correspondant ET type d'appel (maintenant qu'on a filtré les 10 premiers)
grouped_counts = df_top_10.groupby(['CORRESPONDANT', 'TYPE D\'APPEL']).size().reset_index(name='NBRE COMS')

# 6. Ajouter la colonne 'TOTAL_COMS' à grouped_counts
grouped_counts = pd.merge(grouped_counts, correspondant_counts, on='CORRESPONDANT', how='left')

# 7. Calculer le pourcentage du nombre de communications par rapport au nombre total de communications *pour chaque correspondant*
grouped_counts['POURCENTAGE'] = ((grouped_counts['NBRE COMS'] / grouped_counts['TOTAL_COMS']) * 100).round(1)

# 8. Trier grouped_counts par 'TOTAL_COMS' de manière décroissante (important avant de créer le graphique)
grouped_counts = grouped_counts.sort_values(by='TOTAL_COMS', ascending=False)

grouped_counts.head()

,CORRESPONDANT,TYPE D'APPEL,NBRE COMS,TOTAL_COMS,POURCENTAGE
13,262692819895,SMS,214,484,44.2
14,262692819895,TELEPHONIE,270,484,55.8
7,262692485817,SMS,210,241,87.1
8,262692485817,TELEPHONIE,31,241,12.9
9,262692552641,SMS,161,188,85.6


In [91]:

fig.show()fig = px.histogram(grouped_counts, x="CORRESPONDANT", y="NBRE COMS",
             color="TYPE D'APPEL", barmode='group',
             height=400)

In [27]:
len(columns_names)

1

In [ ]:
def count_corr(df):
    try:
        # Compter le nombre de contacts par correspondant
        correspondant_counts = df['CORRESPONDANT'].value_counts().reset_index()
        correspondant_counts.columns = ['CORRESPONDANT', 'NBRE COMS']
        # Filtrer pour garder uniquement les correspondants ayant 11 ou 12 caractères
        correspondant_counts = correspondant_counts[correspondant_counts['CORRESPONDANT'].str.len().isin([11, 12])]
        total_contacts = correspondant_counts['NBRE COMS'].sum()
        # Calculer le POURCENTAGE et arrondir à un chiffre après la virgule
        correspondant_counts['POURCENTAGE'] = ((correspondant_counts['NBRE COMS'] / total_contacts) * 100).round(1)
        # Trier par nombre de contacts du plus élevé au plus bas
        correspondant_counts = correspondant_counts.sort_values(by='NBRE COMS', ascending=False)
        return correspondant_counts
    except Exception as e:
        print(f"Erreur lors du comptage des correspondants: {e}")
        return pd.DataFrame()  # Retourner un DataFrame vide en cas d'erreur

new_df = count_corr(df)

new_df.head()

,CORRESPONDANT,NBRE COMS,POURCENTAGE
1,262692819895,484,38.3
2,262692485817,241,19.1
3,262692552641,188,14.9
4,262692109747,67,5.3
5,262692604166,36,2.9


In [98]:
def plot_correspondant_bar(df):
    try:
        fig = px.bar(df, x='CORRESPONDANT', y='NBRE COMS',
                     title='Nombre de Communications par Correspondant (Top 10)',
                     hover_data=['POURCENTAGE'],  # Inclure TOTAL_COMS dans les informations au survol
                     labels={'CORRESPONDANT': 'Correspondant', 'NBRE COMS': 'Nombre de Communications'})
        # Ajustement de la mise en page
        fig.update_layout(
            xaxis_tickangle=-45,  # Inclinaison des labels de l'axe X

        )
        return fig
    except Exception as e:
        print(f"Erreur lors de la création du graphique à barres: {e}")
        return None

plot_correspondant_bar(new_df.head(10)).show()

In [31]:
import plotly.graph_objects as go
import plotly.express as px

# Créer une carte des adresses pour l'opérateur TCOI.


def carto_adresse_tcoi(df):
    try:
        fig = px.scatter_map(df,
                            lat="Latitude", lon="Longitude",
                            color="Pourcentage", size="Nombre de déclenchement",
                            hover_name="Adresse",
                            size_max=60, zoom=10,
                            color_continuous_scale=px.colors.sequential.Bluered,
                            map_style="carto-positron")

        return fig
    except Exception as e:
        print(f"Erreur lors de la création de la carte des adresses: {e}")
        return go.Figure()

In [32]:
def adresse_count(df):
    try:
       adresse_counts = df['Adresse'].value_counts().reset_index()
       adresse_counts.columns = ['Adresse', 'Nombre de déclenchement']
       adresse_counts.dropna(axis=0, inplace=True)  # Supprimer les lignes sans adresse
       total_contacts = adresse_counts['Nombre de déclenchement'].sum()
       adresse_counts['Pourcentage'] = ((adresse_counts['Nombre de déclenchement'] / total_contacts) * 100).round(1)
       adresse_counts.sort_values(by='Nombre de déclenchement', ascending=False, inplace=True)
       return adresse_counts
    except Exception as e:
       print(f"Erreur lors du comptage des adresses: {e}")
       return pd.DataFrame()

In [33]:
adresse_co = adresse_count(df)
adresse_co.head(50)

,Adresse,Nombre de déclenchement,Pourcentage
0,62 CHEMIN AGAPANTHES 97416 CHALOUPE ST LEU,1435,31.3
1,215 CHEMIN LAURENCY RIVIÈRE 97436 ST LEU,791,17.2
2,INDETERMINE,581,12.7
3,67 RUE HENRI CORNU 97460 ST PAUL,458,10.0
4,633 CHEMIN CENT GAULETTES 97440 ST ANDRE,183,4.0
5,21 AV DES ARTISANS 97416 ST LEU,148,3.2
6,LA SALINE LES HAUTS 97460 ST PAUL,146,3.2
7,24 CHEMIN DES GIROFLÉS - COLIMAÇONS 97419 CHAL...,143,3.1
8,RUELLE DES ZATTES 97416 ST LEU,94,2.0
9,13 CHEMIN MARGUERITES 97426 TROIS BASSINS,79,1.7


In [34]:

new_df = adresse_co.merge(df, on='Adresse', how='left')

In [35]:
carto_adresse_tcoi(new_df)

Erreur lors de la création de la carte des adresses: can only concatenate str (not "int") to str


In [12]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4587 entries, 0 to 4586
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Adresse                  4587 non-null   object        
 1   Nombre de déclenchement  4587 non-null   int64         
 2   Pourcentage              4587 non-null   float64       
 3   Type d'appel             4587 non-null   object        
 4   Abonné                   4587 non-null   object        
 5   Correspondant            4587 non-null   object        
 6   Direction                4587 non-null   object        
 7   Durée                    4587 non-null   object        
 8   IMSI                     4587 non-null   object        
 9   IMEI                     4587 non-null   object        
 10  Ville                    4587 non-null   object        
 11  Latitude                 4006 non-null   object        
 12  Longitude                4006 non-

In [36]:
# Convertir les colonnes en types appropriés si nécessaire
new_df['Latitude'] = pd.to_numeric(new_df['Latitude'], errors='coerce')
new_df['Longitude'] = pd.to_numeric(new_df['Longitude'], errors='coerce')
new_df['Pourcentage'] = pd.to_numeric(new_df['Pourcentage'], errors='coerce')
new_df['Nombre de déclenchement'] = pd.to_numeric(new_df['Nombre de déclenchement'], errors='coerce')

# Supprimer les lignes avec des valeurs manquantes dans les colonnes critiques
new_df.dropna(subset=['Latitude', 'Longitude', 'Pourcentage', 'Nombre de déclenchement'], inplace=True)

In [37]:
carto_adresse_tcoi(new_df)

In [38]:
def carto_adresse_tcoi(df):
    try:
        fig = px.scatter_mapbox(
            df,
            lat="Latitude",
            lon="Longitude",
            color="Pourcentage",
            size="Nombre de déclenchement",
            hover_name="Adresse",
            size_max=60,
            zoom=10,
            color_continuous_scale=px.colors.sequential.Bluered,
            mapbox_style="carto-positron"
        )
        return fig
    except Exception as e:
        st.error(f"Erreur lors de la création de la carte des adresses : {e}")
        return go.Figure()

In [39]:
carto_adresse_tcoi(new_df)